In [2]:
#imports
import numpy as np
import os
import pandas as pd

import math

from datetime import datetime, timedelta
 
import yfinance as yf

import hvplot.pandas

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import panel as pn
pn.config.theme = 'dark'

ImportError: numpy.core.multiarray failed to import

In [ ]:
source_df = None

In [ ]:
symbol = 'QQQ'
file_name = f"{symbol}.csv"
source_df = None

In [ ]:
if os.path.exists(file_name):
    source_df = pd.read_csv(file_name)

In [ ]:

if source_df is None:
    display("download")
    ticker = yf.Ticker(symbol)
    start = datetime.utcnow() - timedelta(days=10*365)
    end=datetime.utcnow() - timedelta(days=1)
    source_df = ticker.history(start=start, end=end)
    source_df.to_csv(f"{symbol}.csv")

In [ ]:
display(source_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
2511,2024-02-22 00:00:00-05:00,434.489990,439.119995,433.709991,438.070007,53887800,0.0,0.0,0.0
2512,2024-02-23 00:00:00-05:00,439.649994,440.589996,435.790009,436.779999,39794400,0.0,0.0,0.0
2513,2024-02-26 00:00:00-05:00,437.600006,438.600006,436.359985,436.549988,33043200,0.0,0.0,0.0
2514,2024-02-27 00:00:00-05:00,437.670013,437.989990,435.029999,437.600006,33713200,0.0,0.0,0.0
2515,2024-02-28 00:00:00-05:00,435.429993,436.670013,434.299988,435.269989,32519656,0.0,0.0,0.0


In [ ]:
bars_df = source_df.copy()
bars_df["Next Close"] = bars_df["Close"].shift(-1)
bars_df["Next High"] = bars_df["High"].shift(-1)
bars_df["Next Low"] = bars_df["Low"].shift(-1)

#15 and 60 minute window for high,low and open
period = 1
windows = [3, 12]

for window in windows:
    wp = window*period
    bars_df[f"High - {wp}"] = bars_df["High"].rolling(window=window).max()
    bars_df[f"Low - {wp}"] = bars_df["Low"].rolling(window=window).min()
    bars_df[f"Open - {wp}"] = bars_df["Open"].shift(periods=window-1)
    
bars_df = bars_df.dropna()
display(bars_df)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,Next High,Next Low,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12
11,2014-03-18 00:00:00-04:00,82.457963,83.452321,82.430343,83.351044,31250200,0.000,0.0,0.0,82.899887,83.470724,82.356670,83.452321,81.601683,81.933136,84.115227,81.601683,82.393501
12,2014-03-19 00:00:00-04:00,83.378648,83.470724,82.356670,82.899887,40869400,0.000,0.0,0.0,83.130066,83.378652,82.476369,83.470724,82.117274,82.163305,84.115227,81.601683,83.461525
13,2014-03-20 00:00:00-04:00,82.642096,83.378652,82.476369,83.130066,31634500,0.000,0.0,0.0,82.129723,83.652354,81.972847,83.470724,82.356670,82.457963,84.115227,81.601683,83.746943
14,2014-03-21 00:00:00-04:00,83.606211,83.652354,81.972847,82.129723,62754000,0.206,0.0,0.0,81.409920,82.489601,80.662449,83.652354,81.972847,83.378648,84.115227,81.601683,84.032347
15,2014-03-24 00:00:00-04:00,82.425005,82.489601,80.662449,81.409920,77735300,0.000,0.0,0.0,81.677574,82.360449,80.994693,83.652354,80.662449,82.642096,84.115227,80.662449,84.032360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,2024-02-21 00:00:00-05:00,424.549988,425.700012,421.630005,425.609985,50179700,0.000,0.0,0.0,438.070007,439.119995,433.709991,434.989990,421.630005,434.890015,439.140015,421.630005,429.070007
2511,2024-02-22 00:00:00-05:00,434.489990,439.119995,433.709991,438.070007,53887800,0.000,0.0,0.0,436.779999,440.589996,435.790009,439.119995,421.630005,428.549988,439.140015,421.630005,429.609985
2512,2024-02-23 00:00:00-05:00,439.649994,440.589996,435.790009,436.779999,39794400,0.000,0.0,0.0,436.549988,438.600006,436.359985,440.589996,421.630005,424.549988,440.589996,421.630005,430.410004
2513,2024-02-26 00:00:00-05:00,437.600006,438.600006,436.359985,436.549988,33043200,0.000,0.0,0.0,437.600006,437.989990,435.029999,440.589996,433.709991,434.489990,440.589996,421.630005,432.109985


In [ ]:
df = bars_df.reset_index()
ys = {
    "Predicted High": df["Next High"], 
    "Predicted Low": df["Next Low"], 
    "Predicted Close": df["Next Close"],
}

X = df.drop(columns=["Next Close", "Next Low", "Next High", "Date"])
display(X)

,index,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12
0,11,82.457963,83.452321,82.430343,83.351044,31250200,0.000,0.0,0.0,83.452321,81.601683,81.933136,84.115227,81.601683,82.393501
1,12,83.378648,83.470724,82.356670,82.899887,40869400,0.000,0.0,0.0,83.470724,82.117274,82.163305,84.115227,81.601683,83.461525
2,13,82.642096,83.378652,82.476369,83.130066,31634500,0.000,0.0,0.0,83.470724,82.356670,82.457963,84.115227,81.601683,83.746943
3,14,83.606211,83.652354,81.972847,82.129723,62754000,0.206,0.0,0.0,83.652354,81.972847,83.378648,84.115227,81.601683,84.032347
4,15,82.425005,82.489601,80.662449,81.409920,77735300,0.000,0.0,0.0,83.652354,80.662449,82.642096,84.115227,80.662449,84.032360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,2510,424.549988,425.700012,421.630005,425.609985,50179700,0.000,0.0,0.0,434.989990,421.630005,434.890015,439.140015,421.630005,429.070007
2500,2511,434.489990,439.119995,433.709991,438.070007,53887800,0.000,0.0,0.0,439.119995,421.630005,428.549988,439.140015,421.630005,429.609985
2501,2512,439.649994,440.589996,435.790009,436.779999,39794400,0.000,0.0,0.0,440.589996,421.630005,424.549988,440.589996,421.630005,430.410004
2502,2513,437.600006,438.600006,436.359985,436.549988,33043200,0.000,0.0,0.0,440.589996,433.709991,434.489990,440.589996,421.630005,432.109985


In [ ]:
for name, y in ys.items():
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    model = ElasticNet()
    model.fit(X_train, y_train)
    predicted_y_values = model.predict(X)
    # create copy of original data
    bars_df[name] = predicted_y_values
    bars_df

/Users/devin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.058e+03, tolerance: 1.985e+03
  model = cd_fast.enet_coordinate_descent(
/Users/devin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+04, tolerance: 1.918e+03
  model = cd_fast.enet_coordinate_descent(
/Users/devin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duali

In [ ]:
display(bars_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Next Low,High - 3,Low - 3,Open - 3,High - 12,Low - 12,Open - 12,Predicted High,Predicted Low,Predicted Close
2510,2024-02-21 00:00:00-05:00,424.549988,425.700012,421.630005,425.609985,50179700,0.0,0.0,0.0,438.070007,...,433.709991,434.989990,421.630005,434.890015,439.140015,421.630005,429.070007,426.122367,419.824447,423.297809
2511,2024-02-22 00:00:00-05:00,434.489990,439.119995,433.709991,438.070007,53887800,0.0,0.0,0.0,436.779999,...,435.790009,439.119995,421.630005,428.549988,439.140015,421.630005,429.609985,439.371526,433.072312,436.447014
2512,2024-02-23 00:00:00-05:00,439.649994,440.589996,435.790009,436.779999,39794400,0.0,0.0,0.0,436.549988,...,436.359985,440.589996,421.630005,424.549988,440.589996,421.630005,430.410004,442.980202,437.026432,440.196697
2513,2024-02-26 00:00:00-05:00,437.600006,438.600006,436.359985,436.549988,33043200,0.0,0.0,0.0,437.600006,...,435.029999,440.589996,433.709991,434.489990,440.589996,421.630005,432.109985,439.695558,434.626729,437.286463
2514,2024-02-27 00:00:00-05:00,437.670013,437.989990,435.029999,437.600006,33713200,0.0,0.0,0.0,435.269989,...,434.299988,440.589996,435.029999,439.649994,440.589996,421.630005,433.940002,439.690066,434.586127,437.241193


In [ ]:
bars_df["High/Low Success"] = np.where(
    (bars_df["Next Close"] >= bars_df["Predicted Low"]),
    np.where(
        bars_df["Next Close"] <= bars_df["Predicted High"], 1, 0
    ), 0
)

In [ ]:
bars_df["Predicted Close Delta"] = bars_df["Predicted Close"] - bars_df["Close"]
bars_df["Next Close Delta"] = bars_df["Next Close"] - bars_df["Close"]
bars_df["Close Product"] = bars_df["Next Close Delta"] * bars_df["Predicted Close Delta"]
bars_df["Close Success"] = np.where(
    bars_df["Close Product"] >= 0, 1, 0
)

In [ ]:
actual_close = bars_df.hvplot.line(
    x="Date",
    y="Next Close",
)

In [ ]:
predicted_high = bars_df.hvplot.line(
    x="Date",
    y="Predicted High",
)

In [ ]:
predicted_low = bars_df.hvplot.line(
    x="Date",
    y="Predicted Low",
)

In [ ]:
predicted_close = bars_df.hvplot.line(
    x="Date",
    y="Predicted Close",
)

In [ ]:
actual_close * predicted_high * predicted_low * predicted_close

:Overlay
   .Curve.I   :Curve   [Date]   (Next Close)
   .Curve.II  :Curve   [Date]   (Predicted High)
   .Curve.III :Curve   [Date]   (Predicted Low)
   .Curve.IV  :Curve   [Date]   (Predicted Close)

In [ ]:
bars_df.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,Next High,...,Low - 12,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success
count,2504.000000,2504.000000,2504.000000,2504.000000,2.504000e+03,2504.000000,2504.0,2504.0,2504.000000,2504.000000,...,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000,2504.000000
mean,207.274857,208.816391,205.622729,207.339951,4.155626e+07,0.006167,0.0,0.0,207.480494,208.957453,...,199.121924,205.736628,208.984854,205.797470,207.513775,0.425719,0.173824,0.140543,0.275735,0.517173
std,101.523363,102.385697,100.614796,101.569227,2.230266e+07,0.050848,0.0,0.0,101.641046,102.456206,...,97.127135,100.735279,102.412343,100.630103,101.535803,0.494550,1.856604,3.269086,10.025895,0.499805
min,77.829482,78.641524,76.851294,77.617241,7.079300e+06,0.000000,0.0,0.0,77.617241,78.641524,...,76.851294,77.829482,77.544098,74.658206,76.132282,0.000000,-15.518196,-22.477631,-81.288016,0.000000
25%,110.449544,110.964894,109.899046,110.519619,2.539732e+07,0.000000,0.0,0.0,110.561167,110.989365,...,107.555085,109.903472,111.199103,110.227786,110.988548,0.000000,-0.556255,-0.853682,-0.582727,0.000000
50%,174.421743,176.001011,173.540460,174.556572,3.622675e+07,0.000000,0.0,0.0,174.705879,176.126226,...,168.108477,173.836399,176.284659,173.388056,174.793232,0.000000,0.203109,0.181908,0.008129,1.000000
75%,301.225266,304.416150,298.234130,302.225227,5.248888e+07,0.000000,0.0,0.0,302.255920,304.706704,...,287.667223,299.928618,304.142830,298.762797,301.508177,1.000000,0.820275,1.333885,0.685005,1.000000
max,439.649994,440.589996,436.359985,438.070007,1.986858e+08,0.808000,0.0,0.0,438.070007,440.589996,...,422.790009,433.940002,442.980202,437.026432,440.196697,1.000000,10.262576,19.250580,115.253311,1.000000


In [ ]:
bars_df["Action"] = np.where(
    bars_df["Predicted Close"] > bars_df["Close"], 1, -1
)

In [ ]:
display(bars_df.head())
display(bars_df.tail())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success,Action
11,2014-03-18 00:00:00-04:00,82.457963,83.452321,82.430343,83.351044,31250200,0.000,0.0,0.0,82.899887,...,82.393501,82.862754,82.233106,82.542153,0,-0.808891,-0.451157,0.364936,1,-1
12,2014-03-19 00:00:00-04:00,83.378648,83.470724,82.356670,82.899887,40869400,0.000,0.0,0.0,83.130066,...,83.461525,83.310646,82.392461,82.837772,1,-0.062115,0.230179,-0.014298,0,-1
13,2014-03-20 00:00:00-04:00,82.642096,83.378652,82.476369,83.130066,31634500,0.000,0.0,0.0,82.129723,...,83.746943,82.856409,82.232775,82.524288,0,-0.605777,-1.000343,0.605985,1,-1
14,2014-03-21 00:00:00-04:00,83.606211,83.652354,81.972847,82.129723,62754000,0.206,0.0,0.0,81.409920,...,84.032347,83.236965,81.597421,82.422682,0,0.292960,-0.719803,-0.210873,0,1
15,2014-03-24 00:00:00-04:00,82.425005,82.489601,80.662449,81.409920,77735300,0.000,0.0,0.0,81.677574,...,84.032360,82.024727,79.761639,80.907587,1,-0.502333,0.267654,-0.134452,0,-1


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Open - 12,Predicted High,Predicted Low,Predicted Close,High/Low Success,Predicted Close Delta,Next Close Delta,Close Product,Close Success,Action
2510,2024-02-21 00:00:00-05:00,424.549988,425.700012,421.630005,425.609985,50179700,0.0,0.0,0.0,438.070007,...,429.070007,426.122367,419.824447,423.297809,0,-2.312176,12.460022,-28.809764,0,-1
2511,2024-02-22 00:00:00-05:00,434.489990,439.119995,433.709991,438.070007,53887800,0.0,0.0,0.0,436.779999,...,429.609985,439.371526,433.072312,436.447014,1,-1.622994,-1.290009,2.093676,1,-1
2512,2024-02-23 00:00:00-05:00,439.649994,440.589996,435.790009,436.779999,39794400,0.0,0.0,0.0,436.549988,...,430.410004,442.980202,437.026432,440.196697,0,3.416698,-0.230011,-0.785878,0,1
2513,2024-02-26 00:00:00-05:00,437.600006,438.600006,436.359985,436.549988,33043200,0.0,0.0,0.0,437.600006,...,432.109985,439.695558,434.626729,437.286463,1,0.736475,1.050018,0.773313,1,1
2514,2024-02-27 00:00:00-05:00,437.670013,437.989990,435.029999,437.600006,33713200,0.0,0.0,0.0,435.269989,...,433.940002,439.690066,434.586127,437.241193,1,-0.358813,-2.330017,0.836040,1,-1


In [ ]:
current_position = 0
starting_cash = 1000
current_cash = starting_cash
max_position = 50
for index, row in bars_df.iterrows():
   
    close = row["Close"]
    action = row["Action"]

    if action > 0:
        if action + current_position <= max_position and action * close < current_cash:
            current_position += action
            current_cash -= action*close
        else:
            action = 0
    elif action < 0:
        if action + current_position >= 0:
            current_position += action
            current_cash += -action*close
        else:
            action = 0

    
    bars_df.loc[index, "Position"] = current_position
    bars_df.loc[index, "Cash"] = current_cash

In [ ]:
bars_df["Holdings"] = bars_df["Close"] * bars_df["Position"]

# profit
bars_df["Strategy Value"] = bars_df["Holdings"] + bars_df["Cash"]
starting_close = bars_df.iloc[0]["Close"] 
display(f"starting_close {starting_close}")
starting_shares = math.floor(starting_cash / starting_close)
display(f"starting_shares {starting_shares}")
bars_df["Stock Value"] =  bars_df["Close"] * starting_shares

# returns
bars_df["Stock Returns"] = bars_df["Close"].pct_change()
bars_df["Strategy Returns"] = bars_df["Strategy Value"].pct_change()

# cumulative daily returns
bars_df["Stock Cumulative Returns"] = (bars_df["Stock Returns"] + 1).cumprod()
bars_df["Strategy Cumulative Returns"] = (bars_df["Strategy Returns"] + 1).cumprod()

'starting_close 83.35104370117188'

'starting_shares 11'

In [ ]:
bars_df.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Next Close,...,Action,Position,Cash,Holdings,Strategy Value,Stock Value,Stock Returns,Strategy Returns,Stock Cumulative Returns,Strategy Cumulative Returns
2510,2024-02-21 00:00:00-05:00,424.549988,425.700012,421.630005,425.609985,50179700,0.0,0.0,0.0,438.070007,...,-1,4.0,2310.043282,1702.439941,4012.483223,4681.709839,-0.004002,-0.002126,5.106235,4.012483
2511,2024-02-22 00:00:00-05:00,434.489990,439.119995,433.709991,438.070007,53887800,0.0,0.0,0.0,436.779999,...,-1,3.0,2748.113289,1314.210022,4062.323311,4818.770081,0.029276,0.012421,5.255723,4.062323
2512,2024-02-23 00:00:00-05:00,439.649994,440.589996,435.790009,436.779999,39794400,0.0,0.0,0.0,436.549988,...,1,4.0,2311.333290,1747.119995,4058.453285,4804.579987,-0.002945,-0.000953,5.240246,4.058453
2513,2024-02-26 00:00:00-05:00,437.600006,438.600006,436.359985,436.549988,33043200,0.0,0.0,0.0,437.600006,...,1,5.0,1874.783302,2182.749939,4057.533241,4802.049866,-0.000527,-0.000227,5.237487,4.057533
2514,2024-02-27 00:00:00-05:00,437.670013,437.989990,435.029999,437.600006,33713200,0.0,0.0,0.0,435.269989,...,-1,4.0,2312.383308,1750.400024,4062.783333,4813.600067,0.002405,0.001294,5.250084,4.062783


In [ ]:
stock_cumulative_returns = bars_df.hvplot.line(
    x="Date",
    y="Stock Cumulative Returns",
)

In [ ]:
strategy_cumulative_returns = bars_df.hvplot.line(
    x="Date",
    y="Strategy Cumulative Returns",
)

In [ ]:
stock_cumulative_returns * strategy_cumulative_returns

:Overlay
   .Curve.I  :Curve   [Date]   (Stock Cumulative Returns)
   .Curve.II :Curve   [Date]   (Strategy Cumulative Returns)

In [ ]:
cash = bars_df.hvplot.line(
    x="Date",
    y="Cash",
)

In [ ]:
holdings = bars_df.hvplot.line(
    x="Date",
    y="Holdings",
)

In [ ]:
cash*holdings

:Overlay
   .Curve.I  :Curve   [Date]   (Cash)
   .Curve.II :Curve   [Date]   (Holdings)

In [ ]:
position = bars_df.hvplot.line(
    x="Date",
    y="Position",
)
position

:Curve   [Date]   (Position)